In [1]:
from bs4 import BeautifulSoup 
from selenium import webdriver
import urllib3
import time 
import random
import requests
import re
import pandas as pd 
import webbrowser
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import tkinter as tk
from tkinter import ttk
import ipywidgets as widgets
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display
import datetime 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from IPython.display import display, HTML
import customtkinter
from tkinter import messagebox
from email.message import EmailMessage
import smtplib

In [ ]:
job_details_links = []

url = "https://fr.indeed.com/jobs?q=data&l=&vjk=23d23734ba242a27"
driver_path = print(ChromeDriverManager().install())  
driver = webdriver.Chrome(driver_path)
driver.get(url)

while True:
    job_postings_elements = driver.find_elements(By.CLASS_NAME, "css-zu9cdh")
    for job in job_postings_elements[:10]:
        for i in range(1, 18):
            if i % 6 == 0:
                continue
            xpath = "/html/body/main/div/div[2]/div/div[5]/div/div[1]/div[5]/div/ul/li[{}]/div/div/div/div/div/table[1]/tbody/tr/td[1]/div[1]/h2/a".format(i)
            element = job.find_elements(By.XPATH, xpath)
            if element:
                job_details_link = element[0].get_attribute('href')
                job_details_links.append(job_details_link)
                
    next_button = driver.find_elements(By.CSS_SELECTOR, "a[data-testid='pagination-page-next']")
    if not next_button:  # If next_button list is empty, there are no more pages
        print("No more pages available.")
        break
    
    next_button_link = next_button[0].get_attribute("href")
    driver.get(next_button_link)

print("Number of job details links found:", len(job_details_links))
print(job_details_links)

In [7]:
def scrape_job_details(url):
    driver_path = print(ChromeDriverManager().install())  
    driver = webdriver.Chrome(driver_path)
    driver.get(url)
    
    # Initializing an empty dictionary to store job details
    job_details = {}           
    
    try:
        # Extracting job title
        job_details['title'] = driver.find_element(By.CLASS_NAME, 'jobsearch-JobInfoHeader-title').text
        
        # Extracting company name
        job_details['company'] = driver.find_element(By.CLASS_NAME, 'css-hon9z8').text

        # Next we would like to extract the location and the postal code of the job posting. However, a few specifications should be made in this regard.
        
        # On Indeed, city and postal code share the same element (whether it be CLASS_NAME, XPATH or other), so we have to separate them. We will do so
        # by identifying if the characters in the location element contain digits, if so, we wil take the digits part and assign it to postal code, and the rest to city.
        location_element = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="inlineHeader-companyLocation"]').text
        if any(char.isdigit() for char in location_element):
            job_details['postal_code'] = ''.join(char for char in location_element if char.isdigit())
            
            # Some postal codes are expressed with 5 numbers, while others are expressed with 2. To garantee a clean and consistent code, we retrun all postal codes to their first 2 numbers
            job_details['postal_code'] = job_details['postal_code'][:2]
            job_details['city'] = ''.join(char for char in location_element if not char.isdigit())
           
            # Replacing parentheses remaining from extracting postal codes with spaces in city if they exist
            if '()' in job_details['city']:
                job_details['city'] = job_details['city'].replace('()', ' ')
        
        # If no digits are found, then we assume there is no postal code specified and we assign the location element directly to city.
        else:
            job_details['postal_code'] = "NA"
            job_details['city'] = location_element
                
        

        # Now, we would like to extract the contract type (CDI, CDD, Alternance, etc) and the salary, IF THEY ARE MENTIONNED!
        try:
            salary_contract_element = driver.find_element(By.CLASS_NAME, 'css-1xkrvql').text
            
            # Once again, salary and contract types on Indeed share the same element, so we will have to separate them. An important notice here is that
            # salaries and contract types, IF they both exist in the element, are always separated by '-' (some jobs metion both, some mention only salaries,
            # some only contract while others don't mention any). Hence, we can easily specify splitting the element by '-' if both elements are present.
            if ' - ' in salary_contract_element:
                salary_contract_parts = salary_contract_element.split(' - ')
                
            # Again, if both salaries and contracts are mentionned, we identify each part simply by identifying wich part of the element is digit (representing salary)
                if any(char.isdigit() for char in salary_contract_parts[0]):
                    job_details['salary'] = salary_contract_parts[0]
                    job_details['contract'] = salary_contract_parts[-1]
                elif any(char.isdigit() for char in salary_contract_parts[-1]):
                    job_details['salary'] = salary_contract_parts[-1]
                    job_details['contract'] = salary_contract_parts[0]

            # If there is no dash (meaning there is either salary or contract but not both, we identify whcih one is mentionned  y scanning for digits in characters.
            # If digits is found, it means only salary is disclosed. If not, it means only contract type is mentionned.
            elif any(char.isdigit() for char in salary_contract_element):
                job_details['salary'] = salary_contract_element
                job_details['contract'] = "Not mentionned"
            elif not any(char.isdigit() for char in salary_contract_element):
                job_details['contract'] = salary_contract_element
                job_details['salary'] = "Not disclosed"
                
        # If we can't find the element to start with, it simply states that neither the salary not the contract type is mentionned.
        except NoSuchElementException:
            job_details['salary'] = "Not disclosed"
            job_details['contract'] = "Not mentionned"

      
        # Extracting if there is possibility to work remotely.
        try:
            remote_work_text = driver.find_element(By.CLASS_NAME, 'css-17cdm7w').text
            job_details['remote_work'] = remote_work_text if remote_work_text else "Not mentioned" 
        except NoSuchElementException:
            job_details['remote_work'] = "Not mentioned"
        # The reason we use both 'try' and 'if' here is that sometimes our driver would not find the element associated with remote_work, so it will
        # return "Not mentionne". But other times, it will find the element but return empty characters. If this is the case, then it should also
        # show "Not mentionned".
    
        
        # Extracting the url associated with the "apply now button"
        try:
            apply_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Continuer pour postuler (s\'ouvre dans un nouvel onglet)"]')
            job_details['url'] = apply_button.get_attribute('href')
        except NoSuchElementException:
            job_details['url'] = url  
            
    except Exception as e:
        print("Error scraping job details from {url}: {str(e)}")
    
    finally:
        driver.quit()  # Close the driver after scraping
    
    return job_details

In [25]:
scrape_job_details('https://fr.indeed.com/viewjob?jk=02aad2e916da2e63&from=serp&vjs=3')

C:\Users\user\.wdm\drivers\chromedriver\win64\123.0.6312.86\chromedriver-win32/chromedriver.exe


{'title': 'Data Scientist H/F',
 'company': 'VIVERIS',
 'postal_code': '67',
 'city': ' Strasbourg',
 'salary': 'De 35 000 € à 50 000 € par an',
 'contract': 'CDI',
 'remote_work': 'Not mentioned',
 'url': 'https://fr.indeed.com/applystart?jk=02aad2e916da2e63&from=vj&pos=bottom&mvj=0&spon=0&sjdu=y6U-JEKHQl-MBcvjxJU-R4DDplm6SwWjHcxyoDIphKk0uTSqenw220lU7jR2vIs9-dkaLYEcCjAOOqYsVVw-Te6JEt2WAQVDrd9XYx6QAYtn3zVwnCEggd6XIxaCh58XfgtuPzI7gxI4HtvCMAlKDAHt7crp9navWOaGUGRdG73XXlKryssZ7Ab76RorgF1Tw9_XINGINv7OnqX9-Wv1cg&vjfrom=serp&asub=mob&astse=80303eba909525a1&assa=641'}

jobs_scraped_correctly = []
jobs_ignored = []
all_job_details = []

# Iterate through each job details link
for job_link in job_details_links[:10]:
    try:
        job_details = scrape_job_details(job_link)
        all_job_details.append(job_details)
        jobs_scraped_correctly.append(job_link)
    except Exception as e:
        print(f"Error scraping job details from {job_link}: {str(e)}")
        jobs_ignored.append(job_link)
        continue

print("Number of jobs successfully extracted:", len(jobs_scraped_correctly))
print("Number of failed jobs extraction:", len(jobs_ignored))      

In [9]:
w = pd.DataFrame(all_job_details)
# Remove duplicate jobs
w = w.drop_duplicates()
w = w.dropna().reset_index(drop=True)
w

,title,company,postal_code,city,salary,contract,remote_work,url
0,Ingénieur Administrateur Systèmes et base de d...,COMMUNAUTE AGGLOMERATION CANNES PAYS DE LERINS,06,Cannes,De 36 000 € à 52 850 € par an,"Temps plein, CDD",Not mentioned,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
1,Lead data scientist - Search & Recommandation ...,Carrefour,91,Massy,Not disclosed,"CDI, Temps plein",Not mentioned,https://fr.indeed.com/applystart?jk=259b988ebd...
2,Master Data Manager (H/F),Limagrain,26,Portes-lès-Valence,Not disclosed,CDI,Télétravail partiel,https://fr.indeed.com/applystart?jk=ac7a49f264...
3,Technicien.ne Configurateur.rice de Données - F/H,RTE - Le Réseau de Transport d'Electricité,13,Marseille,Not disclosed,CDI,Not mentioned,https://fr.indeed.com/applystart?jk=f7108a3073...
4,ALTERNANCE - DATA ENGINEER (F/H) - SEPTEMBRE 2024,Louis Vuitton,75,Paris,Not disclosed,Alternance,Not mentioned,https://fr.indeed.com/applystart?jk=f7d91fc3c1...
5,DATA SCIENTIST - H/F,Generali,93,Saint-Denis,De 45 000 € à 55 000 € par an,"CDI, Temps plein",Télétravail partiel,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
6,Data Engineer (IT) / Freelance,Enterprise Digital Resources Ltd,59,Villeneuve-d'Ascq,De 300 € à 650 € par jour,Indépendant / freelance,Not mentioned,https://fr.indeed.com/applystart?jk=550f876ef6...
7,Data Analyst- H/F/X,Société Générale,92,La Défense,Not disclosed,Alternance,Not mentioned,https://fr.indeed.com/applystart?jk=53118a54c4...
8,STAGE IA/DATA - 4 à 6 MOIS H/F,CNP Assurances,92,Issy-les-Moulineaux,Not disclosed,Temps plein,Not mentioned,https://fr.indeed.com/applystart?jk=f3f7d4a3cd...
9,Tech lead data F/H,Bouygues Telecom,92,Meudon,Not disclosed,"CDI, Temps plein",Not mentioned,https://fr.indeed.com/applystart?jk=165174ec40...


In [10]:
def open_url(event):
    item = event.widget.selection()[0]  # Get the selected item
    url = event.widget.item(item, 'values')[-1]  # Get the URL from the values of the item
    webbrowser.open_new(url)

def display_job_listings(filtered_jobs):
    # Create a new window
    job_window = tk.Toplevel(root)
    job_window.title("Job Listings")

    # Create a Treeview widget to display job listings
    job_tree = ttk.Treeview(job_window)
    job_tree['columns'] = ('company', 'contract', 'city', 'postal_code', 'salary', 'remote_work', 'url')

    # Define column headings
    job_tree.heading('#0', text='Title')
    job_tree.heading('company', text='Company')
    job_tree.heading('contract', text='Contract')
    job_tree.heading('city', text='City')
    job_tree.heading('postal_code', text='Postal Code')
    job_tree.heading('salary', text='Salary')
    job_tree.heading('remote_work', text='Remote Work')
    job_tree.heading('url', text='Go to website')

    # Insert job listings into the Treeview
    for idx, job in filtered_jobs.iterrows():
        title = job['title']
        company = job['company']
        contract = job['contract']
        city = job['city']
        postal_code = job['postal_code']
        salary = job['salary']
        remote_work = job['remote_work']
        url = job['url']

        # Insert job listing into the Treeview
        job_tree.insert('', 'end', text=title, values=(company, contract, city, postal_code, salary, remote_work, url))

    # Bind the event to open the URL when clicked
    job_tree.bind('<ButtonRelease-1>', open_url)

    # Pack the Treeview widget
    job_tree.pack(expand=True, fill=customtkinter.BOTH)

def search_jobs():
    selected_keyword = keyword_var.get()
    selected_contract = contract_var.get()
    selected_city = city_var.get()
    selected_postal_code = postal_code_var.get()
    selected_remote_work = remote_work_var.get()

    # Filter the DataFrame based on user choices            
    filtered_jobs = w.copy()
    
    if selected_keyword != "":
    # Filter based on the keyword in the 'title' column
        filtered_jobs = filtered_jobs[filtered_jobs['title'].str.contains(selected_keyword, case=False) | filtered_jobs['company'].str.contains(selected_keyword, case=False)]
    
    if selected_contract != "Any":
        filtered_jobs = filtered_jobs[filtered_jobs['contract'].str.contains(selected_contract)]
        
    if selected_city != "":
        # Convert both the entered city name and city names in the DataFrame to lowercase
        selected_city_lower = selected_city.lower()
        filtered_jobs = filtered_jobs[filtered_jobs['city'].str.lower().str.contains(selected_city_lower)]
    
    if selected_postal_code != "":
        # Filter based on the entered postal code
        filtered_jobs = filtered_jobs[filtered_jobs['postal_code'] == selected_postal_code]

            
    if selected_remote_work != "Any":
        if selected_remote_work == "Any":
            filtered_jobs = filtered_jobs[filtered_jobs['remote_work'].isin(['Télétravail complet', 'Télétravail partiel', 'Télétravail occasionnel','Not mentionned'])]
        else:
            filtered_jobs = filtered_jobs[filtered_jobs['remote_work'] == selected_remote_work]
   

    # Display the filtered jobs in a new window
    display_job_listings(filtered_jobs)
    


In [11]:
def toggle_entries(x1, x2, y1, y2, z1, z2):
    if similar_offers_var.get() == 1:
        x1.grid(row=7, column=0, padx=5, pady=5, sticky="w")
        x2.grid(row=7, column=1, padx=5, pady=5)
        y1.grid(row=8, column=0, padx=5, pady=5, sticky="w")
        y2.grid(row=8, column=1, padx=5, pady=5)
        z1.grid(row=9, column=0, columnspan=2, padx=5, pady=5,sticky="w")
        z2.grid(row=10, column=1, columnspan=2, padx=5, pady=5)
    else:
        x1.grid_remove()
        x2.grid_remove()
        y1.grid_remove()
        y2.grid_remove()
        z1.grid_remove()
        z2.grid_remove()
        

In [ ]:
# Problem with postal code (Paris 1er)
# Remove "Temp plein" and "Temp partiel" or put them in a sepearate column

In [21]:
# Création de la fenêtre principale
customtkinter.set_appearance_mode("light")
customtkinter.set_default_color_theme("green")

root = customtkinter.CTk()
root.title("Job Search")

# Keyword
keyword_label = customtkinter.CTkLabel(root, text="Keyword:")
keyword_label.grid(row=0, column=0, padx=5, pady=2, sticky="w")
keyword_var = tk.StringVar(root)
keyword_entry = ttk.Entry(root, textvariable=keyword_var)
keyword_entry.grid(row=0, column=1, padx=5, pady=2)

# Contrat
contracts = ["Any", "CDI", "CDD", "Stage", "Alternance","Contrat d'apprentissage","freelance"]  
contract_label = customtkinter.CTkLabel(root, text="Contract:")
contract_label.grid(row=1, column=0, padx=5, pady=2, sticky="w")
contract_var = customtkinter.StringVar(root)
contract_dropdown = ttk.Combobox(root, textvariable=contract_var, values=contracts)
contract_dropdown.grid(row=1, column=1, padx=5, pady=2)
contract_dropdown.current(0)  # Set default selection to "Any"

# Ville
city_label = customtkinter.CTkLabel(root, text="City:")
city_label.grid(row=2, column=0, padx=5, pady=2, sticky="w")
city_var = tk.StringVar(root)
city_entry = ttk.Entry(root, textvariable=city_var)
city_entry.grid(row=2, column=1, padx=5, pady=2)

# Postal Code
postal_code_label = customtkinter.CTkLabel(root, text="Postal Code:")
postal_code_label.grid(row=3, column=0, padx=5, pady=2, sticky="w")
postal_code_var = tk.StringVar(root)
postal_code_entry = ttk.Entry(root, textvariable=postal_code_var)
postal_code_entry.grid(row=3, column=1, padx=5, pady=2)

# Télétravail
remote_work_label = customtkinter.CTkLabel(root, text="Remote Work:")
remote_work_label.grid(row=4, column=0, padx=5, pady=2, sticky="w")
remote_work_var = tk.StringVar(root)
remote_work_options = ['Any','Télétravail complet', 'Télétravail partiel', 'Télétravail occasionnel','Not mentioned']
remote_work_dropdown = ttk.Combobox(root, textvariable=remote_work_var, values=remote_work_options)
remote_work_dropdown.grid(row=4, column=1, padx=5, pady=2)
remote_work_dropdown.current(0)  # Set default selection to "Any"

# Bouton de recherche
search_button = customtkinter.CTkButton(root, text="Search", command=search_jobs)
search_button.grid(row=5, column=0, columnspan=2, padx=5, pady=5)

# Checkbox for receiving similar offers
similar_offers_var = customtkinter.IntVar(root)
similar_offers_checkbox = customtkinter.CTkCheckBox(root, text="Subscribe to our mailing list", variable=similar_offers_var, command=lambda: toggle_entries(name_label, name_entry, email_label, email_entry, save_button, send_alert_button))
similar_offers_checkbox.grid(row=6, column=0, columnspan=2, padx=5, pady=2, sticky="w")

# Enter Name
name_label = customtkinter.CTkLabel(root, text="Enter Name:")
name_label.grid(row=7, column=0, padx=5, pady=2, sticky="w")
name_var = tk.StringVar(root)
name_entry = ttk.Entry(root, textvariable=name_var)
name_entry.grid(row=7, column=1, padx=5, pady=2)

# Enter Email
email_label = customtkinter.CTkLabel(root, text="Enter Email:")
email_label.grid(row=8, column=0, padx=5, pady=2, sticky="w")
email_var = tk.StringVar(root)
email_entry = ttk.Entry(root, textvariable=email_var)
email_entry.grid(row=8, column=1, padx=5, pady=2)

# Button to save criteria

save_criteria_var = customtkinter.IntVar(root)
save_button = customtkinter.CTkCheckBox(root, text="Save Criteria", command=save_criteria)
save_button.grid(row=9, column=0, padx=5, pady=5,sticky="w")

# Button to send job alerts
send_alert_button = customtkinter.CTkButton(root, text="Send Job Alerts", command=send_job_alerts)
send_alert_button.grid(row=10, column=1, columnspan=2, padx=5, pady=5)


toggle_entries(name_label, name_entry, email_label, email_entry, save_button, send_alert_button)

root.mainloop()

KeyboardInterrupt: 

In [18]:
user_criteria = {}

# Function to save user criteria
def save_criteria():
    email = email_var.get()
    criteria = {
        'keyword': keyword_var.get(),
        'contract': contract_var.get(),
        'city': city_var.get(),
        'postal_code': postal_code_var.get(),
        'remote_work': remote_work_var.get()
    }
    user_criteria[email] = criteria
    messagebox.showinfo("Success", "Criteria saved successfully!")

# Function to send job alerts
def send_job_alerts():
    # Get the email address from the entry widget
    email = email_entry.get()

    # Filter job listings based on the criteria
    filtered_jobs = filter_job_listings(w)

    # Check if there are any filtered jobs
    if not filtered_jobs.empty:
        # Convert filtered jobs DataFrame to a formatted string for email body
        jobs_str = ""
        for index, row in filtered_jobs.iterrows():
            job_info = f"Title: {row['title']}\nCompany: {row['company']}\nCity: {row['city']}\nSalary: {row['salary']}\nContract: {row['contract']}\nRemote Work: {row['remote_work']}\nURL: {row['url']}\n\n"
            jobs_str += job_info

        # Send email with job listings
        send_email(email, jobs_str)
        messagebox.showinfo("Success", "Job alerts sent successfully!")
    else:
        messagebox.showinfo("No Jobs Found", "No jobs found based on the criteria.")

def filter_job_listings(dataframe):
    selected_keyword = keyword_var.get()
    selected_contract = contract_var.get()
    selected_city = city_var.get()
    selected_postal_code = postal_code_var.get()
    selected_remote_work = remote_work_var.get()

    # Filter the DataFrame based on user choices
    filtered_jobs = dataframe.copy()

    if selected_keyword != "":
        # Filter based on the keyword in the 'title' column
        filtered_jobs = filtered_jobs[filtered_jobs['title'].str.contains(selected_keyword, case=False) | filtered_jobs['company'].str.contains(selected_keyword, case=False)]

    if selected_contract != "Any":
        # Filter based on the contract type
        filtered_jobs = filtered_jobs[filtered_jobs['contract'].str.contains(selected_contract)]

    if selected_city != "":
        # Convert both the entered city name and city names in the DataFrame to lowercase
        selected_city_lower = selected_city.lower()
        filtered_jobs = filtered_jobs[filtered_jobs['city'].str.lower().str.contains(selected_city_lower)]

    if selected_postal_code != "":
        # Filter based on the entered postal code
        filtered_jobs = filtered_jobs[filtered_jobs['postal_code'] == selected_postal_code]

    if selected_remote_work != "Any":
        filtered_jobs = filtered_jobs[filtered_jobs['remote_work'].str.contains(selected_remote_work)]

    return filtered_jobs


def send_email(email, jobs):
    message = EmailMessage()
    message['from'] = "Job Alert <alouan.test@gmail.com>"
    message['to'] = email
    message['subject'] = "Job Alert: New Job Listings"
    message.set_content(jobs)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login('alouan.test@gmail.com', 'mnep kgfv ttux ybsj')
        server.send_message(message)
        server.quit()


In [22]:
user_criteria

{'aliharb01106@gmail.com': {'keyword': '',
  'contract': 'freelance',
  'city': '',
  'postal_code': '',
  'remote_work': 'Any'},
 'aliharb1106@outlook.com': {'keyword': '',
  'contract': 'CDI',
  'city': '',
  'postal_code': '',
  'remote_work': 'Any'}}

# ali
## harb
### ali

def send_email(email, jobs):
    message = EmailMessage()
    message['from'] = "Job Alert <alouan.test@gmail.com>"
    message['to'] = email
    message['subject'] = "Job Alert: New Job Listings"
    message.set_content(jobs)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login('alouan.test@gmail.com', 'mnep kgfv ttux ybsj')
        server.send_message(message)
        server.quit()
